# Grid search CV

## Setting up

In [1]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# Load data
dataObj = load_breast_cancer()
X = dataObj.data
y = dataObj.target

# Splitting data
X_train, X_test, y_train, y_test = train_test_split(X, y, 
    stratify=y,
    test_size=0.20,
    random_state=1)

In [2]:
from sklearn.svm import SVC

pipe_svc = Pipeline([('scl', StandardScaler()),
            ('clf', SVC(random_state=1))])

In [3]:
# Get parameter names
for k, v in pipe_svc.get_params().items():
    print(f"{k:35.35s}: {str(v):35.35s}...")

memory                             : None                               ...
steps                              : [('scl', StandardScaler()), ('clf',...
verbose                            : False                              ...
scl                                : StandardScaler()                   ...
clf                                : SVC(random_state=1)                ...
scl__copy                          : True                               ...
scl__with_mean                     : True                               ...
scl__with_std                      : True                               ...
clf__C                             : 1.0                                ...
clf__break_ties                    : False                              ...
clf__cache_size                    : 200                                ...
clf__class_weight                  : None                               ...
clf__coef0                         : 0.0                                ...
clf__decisio

In [4]:
param_range = [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0]

set1 = {'clf__C': param_range,
        'clf__kernel': ['linear']}

set2 = {'clf__C': param_range,
        'clf__gamma': param_range,
        'clf__kernel': ['rbf']}

param_grid = [set1, set2]

In [5]:
from sklearn.model_selection import GridSearchCV

gs = GridSearchCV(estimator=pipe_svc, 
                  param_grid=param_grid, 
                  scoring='accuracy', 
                  cv=10,
                  n_jobs=-1)

In [6]:
gs.fit(X_train,y_train)

GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('scl', StandardScaler()),
                                       ('clf', SVC(random_state=1))]),
             n_jobs=-1,
             param_grid=[{'clf__C': [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0,
                                     1000.0],
                          'clf__kernel': ['linear']},
                         {'clf__C': [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0,
                                     1000.0],
                          'clf__gamma': [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0,
                                         100.0, 1000.0],
                          'clf__kernel': ['rbf']}],
             scoring='accuracy')

In [7]:
# Get parameter names
for k, v in gs.get_params().items():
    print(f"{k:35.35s}: {str(v):35.35}...")

cv                                 : 10                                 ...
error_score                        : nan                                ...
estimator__memory                  : None                               ...
estimator__steps                   : [('scl', StandardScaler()), ('clf',...
estimator__verbose                 : False                              ...
estimator__scl                     : StandardScaler()                   ...
estimator__clf                     : SVC(random_state=1)                ...
estimator__scl__copy               : True                               ...
estimator__scl__with_mean          : True                               ...
estimator__scl__with_std           : True                               ...
estimator__clf__C                  : 1.0                                ...
estimator__clf__break_ties         : False                              ...
estimator__clf__cache_size         : 200                                ...
estimator__c

In [14]:
df = pd.DataFrame(gs.cv_results_)
print(df.shape)
display(df.head())

(72, 21)


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_clf__C,param_clf__kernel,param_clf__gamma,params,split0_test_score,split1_test_score,...,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
0,0.009502,0.000806,0.0012,4.000309e-04,0.0001,linear,NaN,"{'clf__C': 0.0001, 'clf__kernel': 'linear'}",0.630435,0.630435,...,0.717391,0.630435,0.644444,0.622222,0.622222,0.622222,0.622222,0.639420,0.027736,27
1,0.006801,0.000600,0.0010,5.378968e-07,0.001,linear,NaN,"{'clf__C': 0.001, 'clf__kernel': 'linear'}",0.956522,0.869565,...,0.956522,0.913043,0.955556,0.933333,0.933333,0.933333,0.933333,0.931932,0.024556,24
2,0.004501,0.000500,0.0010,1.507891e-07,0.01,linear,NaN,"{'clf__C': 0.01, 'clf__kernel': 'linear'}",0.978261,0.956522,...,0.956522,0.956522,0.977778,0.955556,0.977778,0.955556,0.977778,0.964879,0.010637,11
3,0.003901,0.000300,0.0006,4.901351e-04,0.1,linear,NaN,"{'clf__C': 0.1, 'clf__kernel': 'linear'}",0.956522,0.978261,...,0.956522,0.956522,1.000000,0.977778,0.955556,0.933333,1.000000,0.967101,0.020309,8
4,0.004101,0.000300,0.0009,3.000263e-04,1.0,linear,NaN,"{'clf__C': 1.0, 'clf__kernel': 'linear'}",0.934783,0.978261,...,0.978261,0.978261,1.000000,0.977778,0.955556,0.933333,1.000000,0.969275,0.022439,6


In [9]:
df = df.sort_values(by=['rank_test_score'])
display(df.head())

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_clf__C,param_clf__kernel,param_clf__gamma,params,split0_test_score,split1_test_score,...,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
49,0.004201,0.000600,0.0011,3.000738e-04,10.0,rbf,0.001,"{'clf__C': 10.0, 'clf__gamma': 0.001, 'clf__ke...",1.000000,0.978261,...,0.956522,0.978261,0.977778,0.977778,0.977778,0.955556,1.0,0.975845,0.015324,1
56,0.004601,0.001200,0.0010,3.062542e-07,100.0,rbf,0.0001,"{'clf__C': 100.0, 'clf__gamma': 0.0001, 'clf__...",1.000000,0.978261,...,0.956522,0.978261,0.977778,0.977778,0.977778,0.955556,1.0,0.975845,0.015324,1
50,0.003701,0.000458,0.0009,3.000738e-04,10.0,rbf,0.01,"{'clf__C': 10.0, 'clf__gamma': 0.01, 'clf__ker...",0.956522,0.978261,...,0.978261,0.934783,1.000000,0.977778,0.955556,0.977778,1.0,0.971546,0.019611,3
5,0.006101,0.000944,0.0005,5.002261e-04,10.0,linear,NaN,"{'clf__C': 10.0, 'clf__kernel': 'linear'}",0.956522,1.000000,...,0.978261,0.934783,1.000000,0.977778,0.955556,0.955556,1.0,0.971498,0.021971,4
58,0.004001,0.000447,0.0009,3.000895e-04,100.0,rbf,0.01,"{'clf__C': 100.0, 'clf__gamma': 0.01, 'clf__ke...",0.956522,0.978261,...,0.978261,0.934783,0.977778,0.977778,0.977778,0.955556,1.0,0.969324,0.017394,5


In [10]:
print(gs.best_score_)
print(gs.best_params_)

0.9758454106280192
{'clf__C': 10.0, 'clf__gamma': 0.001, 'clf__kernel': 'rbf'}


- Note that grid search already refit the entire training data with the best parameters. You can check this from this setting.


In [11]:
gs.refit

True

In [12]:
y_pred = gs.predict(X_test)
testing_accuracy = gs.score(X_test,y_test)
print(f"Testing accuracy: {testing_accuracy:6.3f}")

Testing accuracy:  0.982


In [13]:
# To do this manually
clf = gs.best_estimator_
clf.fit(X_train, y_train)
testing_accuracy = clf.score(X_test,y_test)
print(f"Testing accuracy: {testing_accuracy:6.3f}")

Testing accuracy:  0.982
